# GROMACS Resume -  Start a long GROMACS run that times out, and resume it

On the supercomputers, a maximum runtime of 24h is enforced. If a longer GROMACS run is needed, the run can easily be resumed. One of the outputs provided by the Rush `gmx` module is designed to be used as the input to the `gmx_resume` module, which will resume the run from the latest checkpoint. The outputs to this module are identical to those of the `gmx` module itself, so the run can be resumed as many times as necessary to finish it.

## 0.0) Imports

In [ ]:
from pathlib import Path
import time

import rush

In [ ]:
# |hide
# Users won't generally create a workspace
# We nuke to ensure run is reproducible
import os

WORK_DIR = Path.home() / "qdx" / "tutorial-gmx-resume"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

2024-04-10 17:21:43,192 - rush - INFO - Not restoring by default via default


In [ ]:
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"
client = rush.build_blocking_provider_with_functions(access_token=RUSH_TOKEN)

2024-04-10 17:21:43,456 - rush - INFO - Not restoring by default via default
2024-04-10 17:21:43,633 - rush - WARNING - Module plip has a different version on the server: github:talo/tengu-plip/1776ec43500276f85b33e53d900388134b00992f#plip_tengu.
                                Use `.update_modules()` to update the lock file
2024-04-10 17:21:43,633 - rush - WARNING - Module auto3d has a different version on the server: github:talo/tengu-auto3d/e1a1a06ea10a6baefea33f2bd0be8c1e0df5d56c#auto3d_tengu.
                                Use `.update_modules()` to update the lock file
2024-04-10 17:21:43,633 - rush - WARNING - Module hermes_energy has a different version on the server: github:talo/tengu-prelude/d14c895b2bc40ab59136382f5b848523bf70436c#hermes_energy.
                                Use `.update_modules()` to update the lock file
2024-04-10 17:21:43,634 - rush - WARNING - Module github:talo/tengu-p2rank/42e6c56a5bfa027d4ca3e8d190b9469b561cf37c#p2rank_tengu is not in the lock file

In [ ]:
# |hide
# We hide this because users will generally not set a workspace, and won't restore by default
client = rush.build_blocking_provider_with_functions(
    batch_tags=["tutorial-resume-gmx"],
    workspace=WORK_DIR,
)

2024-04-10 17:21:44,222 - rush - INFO - Not restoring by default via default


## 0.1) Input Download and Selection

In [ ]:
!pdb_fetch '1B39' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'

## 1) Input Preparation

In [ ]:
_, prepared_protein_pdb = client.prepare_protein(
    Path.cwd() / "1B39_A_nohet.pdb", None, None
)

## 2.1) Run GROMACS (modules: gmx, gmx_resume)
Next we will run a molecular dynamics simulation on our protein using gromacs via the `gmx` module.

We'll set `timeout_duration_mins = 1` so that the run times out before it finishes, and then resume via the `gmx_resume` module using the first output, which is the archive that contains all the necessary data for resuming the run from the last saved checkpoint.

We'll set `checkpoint_interval_mins = 1.0/60` so that the checkpointing takes place once per second.

We can restart as many times as we need to in order for the run to finish. Use a unique tag for each restarted run so that each sequential restart will be tagged and cached appropriately. See below for an example.

For each restarted run, please pass the same config file that was passed to the original `gmx` call. There is no support for shortening or extending runs, or changing any other run parameters, of runs that have already been started. Passing the same config ensures that progress is reported properly and that there are no other inconsistencies. So, the initial run config should specify the full desired run.

One current limitation is that the frame selection can only operate on the data generated by the last call to `gmx` or `gmx_restart`. Otherwise, the output xtc files from all the calls must be joined and processed manually.

In [ ]:
gmx_config = {
    "params_overrides": {
        "nvt": {"nsteps": 2000},
        "npt": {"nsteps": 2000},
        "md": {"nsteps": 150000},
    },
    "frame_sel": {
        "start_time_ps": 290,
        "end_time_ps": 300,
        "delta_time_ps": 1,
    },
    "checkpoint_interval_mins": 1.0 / 60,
    "timeout_duration_mins": 1,
    "num_gpus": 1,
    "save_wets": False,
}

In [ ]:
resume_files_first, streaming_outputs, static_outputs, *rest = client.gmx(
    None,
    prepared_protein_pdb,
    None,
    gmx_config,
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

In [ ]:
resume_files_first.get()

2024-04-10 17:21:47,053 - rush - INFO - Argument ff370371-90b6-4995-8aab-41064ea67fbf is now ModuleInstanceStatus.RESOLVING
2024-04-10 17:22:20,612 - rush - INFO - Argument ff370371-90b6-4995-8aab-41064ea67fbf is now ModuleInstanceStatus.ADMITTED
2024-04-10 17:22:22,778 - rush - INFO - Argument ff370371-90b6-4995-8aab-41064ea67fbf is now ModuleInstanceStatus.DISPATCHED
2024-04-10 17:22:23,853 - rush - INFO - Argument ff370371-90b6-4995-8aab-41064ea67fbf is now ModuleInstanceStatus.RUNNING
2024-04-10 17:23:48,697 - rush - INFO - Argument ff370371-90b6-4995-8aab-41064ea67fbf is now ModuleInstanceStatus.AWAITING_UPLOAD


'https://storage.googleapis.com/qdx-store/58c8901d-6ed7-4a35-a1d5-f7607878f9e1?x-goog-signature=0bfb96d595cdead3286c86597620807aafd6aea903df258a76fc8ee49a31d103c48b3838c8ca11f4394c427a566ce280728b5d46b0a09c5cb6b1e1f75bc6a887988f128378e1de7dfa3af7a41f6bad52f3512ce6d875a7582e36f212fd2275ec7b288b872c41d89ac87ce1fb4f690fa9827a62ea014c10b22e6eeeb2e923704f1dffa000649998f4286cce2d6c33f7dd5669d724d70f3e5966ccf8feff3751f381a68e66e31832a128c4b0644a2f0c51df8a24bbf70cf9537ab5e8da5035fc5cf36b5722017d89a7a8ee7b8c1e1d3188aca2810fa0c26eee4a1cedf89b1c327976a72cd1bed12b706d0701db5c7727389f72163675cda5a6c0e341aca268e794&x-goog-algorithm=GOOG4-RSA-SHA256&x-goog-credential=qdx-store-user%40humming-bird-321603.iam.gserviceaccount.com%2F20240410%2Fasia-southeast1%2Fstorage%2Fgoog4_request&x-goog-date=20240410T092421Z&x-goog-expires=3600&x-goog-signedheaders=host'

## Checking progress of GROMACS run
To determine if your GROMACS run is done, or if further runs are required, you can look at the progress output.
How to check this output is demonstrated below. 


This is an example progress event that indicates that the job has not completed, and will require resuming.
`n` is the number of execution steps in the GMX module. If `n` is less than `n_expected`, or `done` is false the run is not yet completed.


`gmx_resume_tengu progress: {
  "n": 600,
  "n_expected": 601,
  "n_max": 601,
  "done": false
}`

We keep running resume until the progress is done

In [ ]:
help(client.gmx_resume)

Help on function gmx_resume in module rush.provider:

gmx_resume(*args: *tuple[RushObject[bytes], Record], target: 'Target | None' = None, resources: 'Resources | None' = None, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[bytes], RushObject[bytes], RushObject[bytes], RushObject[bytes], RushObject[bytes], RushObject[bytes], RushObject[bytes]]
    Runs a molecular dynamics simluation using GROMACS from either protein, resuming from a checkpoint.
    Uses GMX 2023.3 https://doi.org/10.5281/zenodo.10017686 and Acpype https://doi.org/10.1186/1756-0500-5-367

    Module version:
    `github:talo/tengu-gmx/eaaa2472bd2dc67eed931fa1816fd0b46c509599#gmx_resume_tengu`

    QDX Type Description:

        resume_files: Object[@$Bytes];
        gmx_config: GMXTenguConfig {
            frame_sel: FrameSelConfig {
                delta_time_ps: u32,
                start_time_ps: u32,
                end_time_ps: u32
            }?,
            timeout_duration_m

In [ ]:
done = False
resumes = 0
resume_files = resume_files_first
while not done:
    resume_files, _, _, xtc_dry, pdb_dry, _, _ = client.gmx_resume(
        resume_files,
        gmx_config,
        tags=[f"gmx-resume-{resumes}"],
        restore=False,
        resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
    )
    # wait for module to finish
    resume_files.get()
    progress = client.module_instance_blocking(resume_files.source).progress
    print(progress)
    done = progress.done
    resumes += 1

2024-04-10 17:24:22,169 - rush - INFO - Argument e8a349bc-a6ab-4753-ba70-b1bd867eaa1f is now ModuleInstanceStatus.RESOLVING
2024-04-10 17:24:23,277 - rush - INFO - Argument e8a349bc-a6ab-4753-ba70-b1bd867eaa1f is now ModuleInstanceStatus.ADMITTED
2024-04-10 17:24:29,853 - rush - INFO - Argument e8a349bc-a6ab-4753-ba70-b1bd867eaa1f is now ModuleInstanceStatus.RUNNING
2024-04-10 17:25:00,050 - rush - INFO - Argument e8a349bc-a6ab-4753-ba70-b1bd867eaa1f is now ModuleInstanceStatus.AWAITING_UPLOAD
n=154000 n_expected=154000 n_max=154000 done=True


# Downloading Results
To download extracted frames and fetch their pdbs, we can do the following

In [ ]:
pdb_dry.download("dry_frames.tar.gz")

PosixPath('/home/machineer/qdx/tutorial-gmx-resume/objects/dry_frames.tar.gz')

In [ ]:
pdb_dry.value

{'path': '045bb474-ba46-4be6-b685-2192bd670074', 'size': 0}

In [ ]:
import tarfile

with tarfile.open(client.workspace / "objects" / "dry_frames.tar.gz", "r") as tf:
    selected_frame_pdbs = [
        tf.extractfile(member).read()
        for member in tf
        if "pdb" in member.name and member.isfile()
    ]
    for i, frame in enumerate(selected_frame_pdbs):
        with open(
            client.workspace / "objects" / f"gmx_output_frame_{i}.pdb", "w"
        ) as pf:
            print(frame.decode("utf-8"), file=pf)

In [ ]:
with open(client.workspace / "objects" / "gmx_output_frame_0.pdb", "r") as f:
    print(str.join("", f.readlines()[0:10]))

REMARK    GENERATED BY TRJCONV
TITLE     Protein in water t= 295.00000 step= 147500
REMARK    THIS IS A SIMULATION BOX
CRYST1   87.777   87.777   87.777  60.00  60.00  90.00 P 1           1
MODEL        6
ATOM      1  N   MET     1      50.060  68.710   7.380  1.00  0.00           N
ATOM      2  H1  MET     1      49.560  67.830   7.380  1.00  0.00           H
ATOM      3  H2  MET     1      49.990  69.010   8.340  1.00  0.00           H
ATOM      4  H3  MET     1      49.660  69.430   6.810  1.00  0.00           H
ATOM      5  CA  MET     1      51.550  68.700   7.170  1.00  0.00           C

